<a href="https://colab.research.google.com/github/PhillySchoolofAI/DL-Libraries/blob/master/Keras_LSTM_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Tutorial - [How to train Keras model x20 times faster with TPU for free](https://www.dlology.com/blog/how-to-train-keras-model-x20-times-faster-with-tpu-for-free/)

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding

In [0]:

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words (among top max_features most common words)
maxlen = 500

# Load data
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Reverse sequences
x_train = [x[::-1] for x in x_train]
x_test = [x[::-1] for x in x_test]

# Pad sequences
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [0]:
def make_model(batch_size=None):
  source = Input(shape=(maxlen,), batch_size=batch_size, dtype=tf.int32, name='Input')
  embedding = Embedding(input_dim=max_features, output_dim=128, name='Embedding')(source)
  # lstm = Bidirectional(LSTM(32, name = 'LSTM'), name='Bidirectional')(embedding)
  lstm = LSTM(32, name = 'LSTM')(embedding)
  predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
  model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
  model.compile(
      optimizer=tf.train.RMSPropOptimizer(learning_rate=0.01),
      loss='binary_crossentropy',
      metrics=['acc'])
  return model
   

In [0]:
tf.keras.backend.clear_session()
training_model = make_model(batch_size = 128)
training_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (128, 500)                0         
_________________________________________________________________
Embedding (Embedding)        (128, 500, 128)           1280000   
_________________________________________________________________
LSTM (LSTM)                  (128, 32)                 20608     
_________________________________________________________________
Output (Dense)               (128, 1)                  33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


In [0]:
import os
# This address identifies the TPU we'll use when configuring TensorFlow.
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    training_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

tpu_model.summary()

INFO:tensorflow:Querying Tensorflow master (b'grpc://10.102.168.98:8470') for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 18360116929208696288)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2556736422492173676)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 1785311708648481474)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7274077856663094447)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 1661440383431074741)
INFO:tensorflow:*** Available Device: _Device

In [0]:
import time
start_time = time.time()

history = tpu_model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128 * 8,
                    validation_split=0.2)
tpu_model.save_weights('./tpu_model.h5', overwrite=True)

print("--- %s seconds ---" % (time.time() - start_time))


Train on 25000 samples, validate on 5000 samples
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(128, 500), dtype=tf.int32, name='Input0'), TensorSpec(shape=(128, 1), dtype=tf.float32, name='Output_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 7.792444467544556 secs
INFO:tensorflow:Setting weights on TPU model.
23552/25000 [===========================>..] - ETA: 0s - loss: 0.6932 - acc: 0.4854INFO:tensorflow:New input shapes; (re-)compiling: mode=train, [TensorSpec(shape=(53, 500), dtype=tf.int32, name='Input0'), TensorSpec(shape=(53, 1), dtype=tf.float32, name='Output_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 7.856823205947876 secs
24576/25000 [====

In [0]:
# history = tpu_model.fit(x_train, y_train,
#                     epochs=20,
#                     batch_size=128 * 8,
#                     validation_split=0.2)
# tpu_model.save_weights('./tpu_model.h5', overwrite=True)

In [0]:
inferencing_model = make_model(batch_size=None)
inferencing_model.load_weights('./tpu_model.h5')
inferencing_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 500)               0         
_________________________________________________________________
Embedding (Embedding)        (None, 500, 128)          1280000   
_________________________________________________________________
LSTM (LSTM)                  (None, 32)                20608     
_________________________________________________________________
Output (Dense)               (None, 1)                 33        
Total params: 1,300,641
Trainable params: 1,300,641
Non-trainable params: 0
_________________________________________________________________


In [0]:
inferencing_model.evaluate(x_test, y_test)

25000/25000 [==============================] - 83s 3ms/step


[0.6060782189846039, 0.824]

In [0]:
tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)

24576/25000 [============================>.] - ETA: 0sINFO:tensorflow:New input shapes; (re-)compiling: mode=eval, [TensorSpec(shape=(53, 500), dtype=tf.int32, name='Input0'), TensorSpec(shape=(53, 1), dtype=tf.float32, name='Output_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 8.719648122787476 secs
25000/25000 [==============================] - 12s 482us/step


[0.608636932144165, 0.8271999996948243]

In [0]:
tpu_model.evaluate(x_test, y_test, batch_size=128 * 8)

25000/25000 [==============================] - 2s 94us/step


[0.608636932144165, 0.8271999996948243]

In [0]:
inferencing_model.predict(x_test[:10])> 0.5

array([[False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True]])

In [0]:
y_test[:10]

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1])

In [0]:
tpu_model.predict_on_batch(x_train[:128 * 8])>0.5

INFO:tensorflow:New input shapes; (re-)compiling: mode=infer, [TensorSpec(shape=(128, 500), dtype=tf.int32, name='Input0')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 7.4768149852752686 secs


array([[ True],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

## Download the trained model weights to your local file system

In [0]:
from google.colab import files

files.download('./tpu_model.h5')